In [1]:
from flask import Flask, request, send_from_directory
from flask_restful import Resource, Api
from PIL import Image
import base64
import io
import os

app = Flask(__name__)
api = Api(app)

# Ensure the 'images' directory exists
if not os.path.exists('images'):
    os.makedirs('images')

class ImageResource(Resource):
    # 이 부분은 인공지능 결과에 대한 데이터가 보내져야 함
    def get(self, image_name):
        if os.path.isfile(os.path.join('images', image_name)):
            return send_from_directory('images', image_name)
        else:
            return {'error': 'Image not found'}, 404

    # 이미지를 받으면, 저장하고, 인공지능 모델에 넣어야 함
    def post(self):
        data = request.get_json()
        image_data = data.get('image', None)
        if image_data:
            try:
                image_data = base64.b64decode(image_data)
                image = Image.open(io.BytesIO(image_data))
                image_name = data.get('name', 'unnamed.jpg')
                image_path = os.path.join('images', image_name)
                image.save(image_path)


                
                return {'message': 'Image saved successfully', 'name': image_name}
            
            except Exception as e:
                return {'error': str(e)}, 400
        else:
            return {'error': 'No image data found'}, 400

api.add_resource(ImageResource, '/image', '/image/<string:image_name>')

In [3]:
import base64
import json

# 이미지를 base64로 인코딩
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
    return encoded_string

# 이미지 파일 경로
image_path = "images/test.png"

# 이미지 인코딩
encoded_image = encode_image(image_path)

# JSON 형태로 변환
json_data = {
    "name": "example.jpg",
    "image": encoded_image
}

# JSON 문자열로 변환
json_string = json.dumps(json_data)

print(json_string)

{"name": "example.jpg", "image": "iVBORw0KGgoAAAANSUhEUgAAAMgAAADICAIAAAAiOjnJAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAAEnQAABJ0Ad5mH3gAAAhDSURBVHhe7d2tchsxFIZhT1FgYaEvobAw0DAwsDCwMMywlxAYWFhoaBhoWFhoGGjYnrG+aDRxbOtnT7RZvw/oeLWRkll9XWl/PfsHOCBYcEGw4IJgwQXBgguCBRcECy4IFlwQLLggWHBBsOCCYMEFwYILggUXBAsuCBZcECy4IFhwQbDggmDBBcGCC4IFFwQLLggWXBAsuCBYcEGw4IJgwQXBgguCBRcECy4IFlwQLLggWHBBsOCCYMEFwYILggUXBAsuCBZcECy4IFhwQbDggmDBRYdgPT4+Xl9f279axhS9a7AsTPP5fLZ3dXW12+20ApPzfsF6eHgIkQpub2+1AlP0TsHabre2i1Km9qlidzVt7xSsu7u7EKmvX78SqUvwHsFKB8Hfv3+rFJPmHqx0EFwsFirF1LkH68ePHyFVDIIXxT1YX758CcFiELwovsFar9chVfP5XEW4DL7BigeD9/f3KsJlcAxWOm1/enpSKS6DY7DSabuKTlqtVlxDnAyvYP39+zfurjKn7WGab7W0jI/MK1jfvn0LqbIPKjon/LzRMj4yl178+fNniIjtfv78+aPSc0IVo+UqjKcjMXyw0jm7JUylGUIVo+UqjKcjMXywSufsUahltFxFTTCe9jZwB1TM2aNQy2i5ipogWL0N3AEVc/YoVDRarqImCFZvQ3ZA3Zw9CnVNywRcTRCs3obsgHi9uWjOHoW6pmUCHlowWkYnQ3aAurS2U1U5oRUlVJNg9Uaw4GKMwYrHlcvlUuuyhYpGy+hkjMGKZ8LMZrPR6jyqRrB6G6wDttuturQ5WLvdzo4Kw+ebmxutzhNqGS2jk8E6IH3AS0WFQnVjn2

In [4]:
encoded_image

'iVBORw0KGgoAAAANSUhEUgAAAMgAAADICAIAAAAiOjnJAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAAEnQAABJ0Ad5mH3gAAAhDSURBVHhe7d2tchsxFIZhT1FgYaEvobAw0DAwsDCwMMywlxAYWFhoaBhoWFhoGGjYnrG+aDRxbOtnT7RZvw/oeLWRkll9XWl/PfsHOCBYcEGw4IJgwQXBgguCBRcECy4IFlwQLLggWHBBsOCCYMEFwYILggUXBAsuCBZcECy4IFhwQbDggmDBBcGCC4IFFwQLLggWXBAsuCBYcEGw4IJgwQXBgguCBRcECy4IFlwQLLggWHBBsOCCYMEFwYILggUXBAsuCBZcECy4IFhwQbDggmDBRYdgPT4+Xl9f279axhS9a7AsTPP5fLZ3dXW12+20ApPzfsF6eHgIkQpub2+1AlP0TsHabre2i1Km9qlidzVt7xSsu7u7EKmvX78SqUvwHsFKB8Hfv3+rFJPmHqx0EFwsFirF1LkH68ePHyFVDIIXxT1YX758CcFiELwovsFar9chVfP5XEW4DL7BigeD9/f3KsJlcAxWOm1/enpSKS6DY7DSabuKTlqtVlxDnAyvYP39+zfurjKn7WGab7W0jI/MK1jfvn0LqbIPKjon/LzRMj4yl178+fNniIjtfv78+aPSc0IVo+UqjKcjMXyw0jm7JUylGUIVo+UqjKcjMXywSufsUahltFxFTTCe9jZwB1TM2aNQy2i5ipogWL0N3AEVc/YoVDRarqImCFZvQ3ZA3Zw9CnVNywRcTRCs3obsgHi9uWjOHoW6pmUCHlowWkYnQ3aAurS2U1U5oRUlVJNg9Uaw4GKMwYrHlcvlUuuyhYpGy+hkjMGKZ8LMZrPR6jyqRrB6G6wDttuturQ5WLvdzo4Kw+ebmxutzhNqGS2jk8E6IH3AS0WFQnVjn21HpYVa8/mcK4Yd1QdrtVqF5+Xt33he1FT

In [ ]:
image_data = base64.b64decode(encoded_image)
image = Image.open(io.BytesIO(image_data))
image_name = json_data.get('name', 'unnamed.jpg')
image_path = os.path.join('images', image_name)
image.save(image_path)

In [6]:
json_data.get('name', 'unnamed.jpg')

'example.jpg'

In [7]:
json.loads(json_string)

{'name': 'example.jpg',
 'image': 'iVBORw0KGgoAAAANSUhEUgAAAMgAAADICAIAAAAiOjnJAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAAEnQAABJ0Ad5mH3gAAAhDSURBVHhe7d2tchsxFIZhT1FgYaEvobAw0DAwsDCwMMywlxAYWFhoaBhoWFhoGGjYnrG+aDRxbOtnT7RZvw/oeLWRkll9XWl/PfsHOCBYcEGw4IJgwQXBgguCBRcECy4IFlwQLLggWHBBsOCCYMEFwYILggUXBAsuCBZcECy4IFhwQbDggmDBBcGCC4IFFwQLLggWXBAsuCBYcEGw4IJgwQXBgguCBRcECy4IFlwQLLggWHBBsOCCYMEFwYILggUXBAsuCBZcECy4IFhwQbDggmDBRYdgPT4+Xl9f279axhS9a7AsTPP5fLZ3dXW12+20ApPzfsF6eHgIkQpub2+1AlP0TsHabre2i1Km9qlidzVt7xSsu7u7EKmvX78SqUvwHsFKB8Hfv3+rFJPmHqx0EFwsFirF1LkH68ePHyFVDIIXxT1YX758CcFiELwovsFar9chVfP5XEW4DL7BigeD9/f3KsJlcAxWOm1/enpSKS6DY7DSabuKTlqtVlxDnAyvYP39+zfurjKn7WGab7W0jI/MK1jfvn0LqbIPKjon/LzRMj4yl178+fNniIjtfv78+aPSc0IVo+UqjKcjMXyw0jm7JUylGUIVo+UqjKcjMXywSufsUahltFxFTTCe9jZwB1TM2aNQy2i5ipogWL0N3AEVc/YoVDRarqImCFZvQ3ZA3Zw9CnVNywRcTRCs3obsgHi9uWjOHoW6pmUCHlowWkYnQ3aAurS2U1U5oRUlVJNg9Uaw4GKMwYrHlcvlUuuyhYpGy+hkjMGKZ8LMZrPR6jyqRrB6G6wDttuturQ5WLvdzo4Kw+ebmxutzhNqGS2jk8E6IH3AS0WFQnVjn

In [9]:
!pip install requests

  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Using cached urllib3-1.26.15-py2.py3-none-any.whl (140 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached charset_normalizer-3.1.0-cp39-cp39-win_amd64.whl (97 kB)


In [17]:
import requests

url = "http://192.168.122.62:5000/images/example.jpg"

http_post_request = requests.get(url, params={"name": "user1"})
print(http_post_request.text)
print(http_post_request.status_code)

{
    "message": "The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again."
}

404


In [15]:
json_data

{'name': 'example.jpg',
 'image': 'iVBORw0KGgoAAAANSUhEUgAAAMgAAADICAIAAAAiOjnJAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAAEnQAABJ0Ad5mH3gAAAhDSURBVHhe7d2tchsxFIZhT1FgYaEvobAw0DAwsDCwMMywlxAYWFhoaBhoWFhoGGjYnrG+aDRxbOtnT7RZvw/oeLWRkll9XWl/PfsHOCBYcEGw4IJgwQXBgguCBRcECy4IFlwQLLggWHBBsOCCYMEFwYILggUXBAsuCBZcECy4IFhwQbDggmDBBcGCC4IFFwQLLggWXBAsuCBYcEGw4IJgwQXBgguCBRcECy4IFlwQLLggWHBBsOCCYMEFwYILggUXBAsuCBZcECy4IFhwQbDggmDBRYdgPT4+Xl9f279axhS9a7AsTPP5fLZ3dXW12+20ApPzfsF6eHgIkQpub2+1AlP0TsHabre2i1Km9qlidzVt7xSsu7u7EKmvX78SqUvwHsFKB8Hfv3+rFJPmHqx0EFwsFirF1LkH68ePHyFVDIIXxT1YX758CcFiELwovsFar9chVfP5XEW4DL7BigeD9/f3KsJlcAxWOm1/enpSKS6DY7DSabuKTlqtVlxDnAyvYP39+zfurjKn7WGab7W0jI/MK1jfvn0LqbIPKjon/LzRMj4yl178+fNniIjtfv78+aPSc0IVo+UqjKcjMXyw0jm7JUylGUIVo+UqjKcjMXywSufsUahltFxFTTCe9jZwB1TM2aNQy2i5ipogWL0N3AEVc/YoVDRarqImCFZvQ3ZA3Zw9CnVNywRcTRCs3obsgHi9uWjOHoW6pmUCHlowWkYnQ3aAurS2U1U5oRUlVJNg9Uaw4GKMwYrHlcvlUuuyhYpGy+hkjMGKZ8LMZrPR6jyqRrB6G6wDttuturQ5WLvdzo4Kw+ebmxutzhNqGS2jk8E6IH3AS0WFQnVjn

In [16]:
requests.post(url, json=json_data, headers={'Content-Type': 'application/json; charset=utf-8'})

<Response [200]>